In [1]:
install.packages("remotes")
remotes::install_github("cellgeni/sceasy")

Updating HTML index of packages in '.Library'

Making 'packages.html' ...
 done

Skipping install of 'sceasy' from a github remote, the SHA1 (0cfc0e39) has not changed since last install.
  Use `force = TRUE` to force installation



In [2]:
library(sceasy)
suppressPackageStartupMessages(library(DESeq2))
suppressPackageStartupMessages(library(H5weaver))
suppressPackageStartupMessages(library(dplyr))
suppressPackageStartupMessages(library(tidyr))
suppressPackageStartupMessages(library(tibble))
suppressPackageStartupMessages(library(ggplot2))
suppressPackageStartupMessages(library(Seurat))
suppressPackageStartupMessages(library(stringr))
library(parallel)

Loading required package: reticulate



In [3]:
meta_data=read.csv('/home/jupyter/formal_BRIanalysis/DEG_Mehul/hise_meta_data_2024-01-23_fixed.csv')
meta_data<-meta_data %>% filter(Covid_exlcusion=='no',sample.visitName %in% c('Flu Year 1 Day 0'))
meta_data<-meta_data %>% mutate (Flu_Year=case_when(Sample_Draw_date>as.Date('2019-6-30')&Sample_Draw_date<as.Date('2020-6-29')~'2019',
                                         Sample_Draw_date>as.Date('2020-6-30')&Sample_Draw_date<as.Date('2021-6-29')~'2020',
                                         Sample_Draw_date>as.Date('2021-6-30')&Sample_Draw_date<as.Date('2022-6-29')~'2021',
                                        ))

In [4]:
t(table(meta_data$subject.subjectGuid,meta_data$Flu_Year))

      
       BR1001 BR1002 BR1003 BR1004 BR1005 BR1006 BR1007 BR1008 BR1009 BR1010
  2019      1      1      1      1      1      1      1      1      1      1
  2020      0      0      0      0      0      0      0      0      0      0
      
       BR1011 BR1012 BR1013 BR1014 BR1015 BR1016 BR1017 BR1018 BR1019 BR1021
  2019      1      1      1      1      0      0      0      1      0      0
  2020      0      0      0      0      1      1      1      0      1      1
      
       BR1023 BR1024 BR1025 BR1026 BR1028 BR1030 BR1031 BR1032 BR1033 BR1037
  2019      0      0      0      0      0      0      0      0      1      0
  2020      1      1      1      1      1      1      1      1      0      1
      
       BR1040 BR1041 BR1042 BR1043 BR1044 BR1045 BR1047 BR1048 BR1049 BR1050
  2019      0      0      0      0      0      0      0      0      0      0
  2020      1      1      1      1      1      1      1      1      1      1
      
       BR1051 BR1052 BR1054 BR1056 BR1057

# Year 1 Day 0, compare male vs female

### Y1

In [5]:
meta_data_subset<-meta_data%>% filter(sample.visitName %in% c('Flu Year 1 Day 0'))

In [8]:
system.time(
bulk_list<-mclapply(meta_data_subset$pbmc_sample_id,function(x){

pbmc <-sceasy::convertFormat(paste0('/home/jupyter/h5_cleaned_by_sample/',x,'.h5ad'), from="anndata", to="seurat")
bulk <- Seurat:::PseudobulkExpression(object = pbmc, pb.method = 'aggregate',   group.by = "AIFI_L3",slot = 'counts')$RNA
colnames(bulk)<-paste0(x,":",colnames(bulk))
return(bulk)
},mc.cores=60)
)

   user  system elapsed 
879.604 374.130 254.277 

In [12]:
system.time(
for (celltype in str_extract(colnames(bulk_list[[1]]), "(?<=:).*")){
print(celltype)
matirx_list<-mclapply(bulk_list,function(bulk_matrix){

matching_columns <- grepl(celltype, colnames(bulk_matrix),, fixed = TRUE)
matching_columns<-colnames(bulk_matrix)[matching_columns]
bulk_matrix_single <- as.data.frame(bulk_matrix[, matching_columns, drop = FALSE])
return(bulk_matrix_single)
},mc.cores=60)

df_all<-do.call(cbind,matirx_list)
df_meta_data<-as.data.frame(colnames(df_all))
colnames(df_meta_data)<-'sample_name'
split_columns <- strsplit(df_meta_data$sample_name, ":")
df_meta_data$pbmc_sample_id <- sapply(split_columns, `[`, 1)
df_meta_data$AIFI_L3 <- sapply(split_columns, `[`, 2)
df_meta_data<-left_join(df_meta_data,meta_data_subset)
rownames(df_meta_data)<-df_meta_data$sample_name
selected_gene_list<-read.csv('/home/jupyter/formal_BRIanalysis/DEG_Mehul/filtered_gene_list_for_sex.csv')
selGenes<-selected_gene_list %>% filter(AIFI_L3==celltype) %>% select(gene) %>% pull()  

dds <- DESeqDataSetFromMatrix(df_all[selGenes,], colData = df_meta_data, 
                              design = ~  subject.biologicalSex+
                                          cohort.cohortGuid+
                                          CMV.IgG.Serology.Result.Interpretation)
dds <- DESeq(dds,parallel=FALSE)

res <- data.frame(results(dds,contrast=c("subject.biologicalSex", "Male", "Female")))
res$celltype<-celltype
res=res %>% arrange(padj) %>% mutate(Direction=case_when(log2FoldChange<0~'Female',log2FoldChange>0~'Male'))
res$Year<-"Y1"
write.csv(res,paste0("/home/jupyter/formal_BRIanalysis/DEG_Mehul/Sex_Year1Day0/",celltype,"_Y1D0.csv"))

}
)

[1] "ASDC"


Joining with `by = join_by(pbmc_sample_id)`
converting counts to integer mode

Warning message in DESeqDataSet(se, design = design, ignoreRank):
“some variables in design formula are characters, converting to factors”
estimating size factors

estimating dispersions

gene-wise dispersion estimates

mean-dispersion relationship

final dispersion estimates

fitting model and testing



[1] "Activated memory B cell"


Joining with `by = join_by(pbmc_sample_id)`
converting counts to integer mode

Warning message in DESeqDataSet(se, design = design, ignoreRank):
“some variables in design formula are characters, converting to factors”
estimating size factors

estimating dispersions

gene-wise dispersion estimates

mean-dispersion relationship

final dispersion estimates

fitting model and testing



[1] "Adaptive NK cell"


...

[1] "cDC1"


Joining with `by = join_by(pbmc_sample_id)`
converting counts to integer mode

Warning message in DESeqDataSet(se, design = design, ignoreRank):
“some variables in design formula are characters, converting to factors”
estimating size factors

estimating dispersions

gene-wise dispersion estimates

mean-dispersion relationship

final dispersion estimates

fitting model and testing



[1] "pDC"


Joining with `by = join_by(pbmc_sample_id)`
converting counts to integer mode

Warning message in DESeqDataSet(se, design = design, ignoreRank):
“some variables in design formula are characters, converting to factors”
estimating size factors

estimating dispersions

gene-wise dispersion estimates

mean-dispersion relationship

final dispersion estimates

fitting model and testing



    user   system  elapsed 
1382.133 1678.268 1126.528 

### Y2

In [9]:
meta_data_subset<-meta_data%>% filter(sample.visitName %in% c('Flu Year 2 Day 0','Flu Year 2 Day 7'))

In [12]:
bulk_list<-mclapply(meta_data_subset$pbmc_sample_id,function(x){

pbmc <-sceasy::convertFormat(paste0('/home/jupyter/BRI_Analysis/scRNA/h5_cleaned_by_sample/',x,'.h5ad'), from="anndata", to="seurat")
bulk <- Seurat:::PseudobulkExpression(object = pbmc, pb.method = 'aggregate',   group.by = "AIFI_L3",slot = 'counts')$RNA
colnames(bulk)<-paste0(x,":",colnames(bulk))
return(bulk)
},mc.cores=60)


In [13]:
for (celltype in str_extract(colnames(bulk_list[[1]]), "(?<=:).*")){
print(celltype)
if (celltype=='Erythrocyte'){next}
matirx_list<-mclapply(bulk_list,function(bulk_matrix){

matching_columns <- grepl(celltype, colnames(bulk_matrix),, fixed = TRUE)
matching_columns<-colnames(bulk_matrix)[matching_columns]
bulk_matrix_single <- as.data.frame(bulk_matrix[, matching_columns, drop = FALSE])
return(bulk_matrix_single)
},mc.cores=60)

df_all<-do.call(cbind,matirx_list)
df_meta_data<-as.data.frame(colnames(df_all))
colnames(df_meta_data)<-'sample_name'
split_columns <- strsplit(df_meta_data$sample_name, ":")
df_meta_data$pbmc_sample_id <- sapply(split_columns, `[`, 1)
df_meta_data$AIFI_L3 <- sapply(split_columns, `[`, 2)
df_meta_data<-left_join(df_meta_data,meta_data_subset)
rownames(df_meta_data)<-df_meta_data$sample_name
selected_gene_list<-read.csv('filtered_gene_list.csv')
selGenes<-selected_gene_list %>% filter(AIFI_L3==celltype) %>% select(gene) %>% pull()  
    
dds <- DESeqDataSetFromMatrix(df_all[selGenes,], colData = df_meta_data, 
                              design = ~  sample.visitName+
                                          subject.biologicalSex+cohort.cohortGuid+
                                          CMV.IgG.Serology.Result.Interpretation)
dds <- DESeq(dds,parallel=FALSE)

res <- data.frame(results(dds,contrast=c("sample.visitName", "Flu Year 2 Day 0", "Flu Year 2 Day 7")))
res$celltype<-celltype
res=res %>% arrange(padj) %>% mutate(Direction=case_when(log2FoldChange<0~'HigherInD7',log2FoldChange>0~'HigherInD0'))
res$Year<-"Y2"
write.csv(res,paste0("01C_DESeq2_D0_VS_D7_L3/",celltype,"_Y2.csv"))

}

[1] "ASDC"


Joining with `by = join_by(pbmc_sample_id)`
converting counts to integer mode

Warning message in DESeqDataSet(se, design = design, ignoreRank):
“some variables in design formula are characters, converting to factors”
  Note: levels of factors in the design contain characters other than
  letters, numbers, '_' and '.'. It is recommended (but not required) to use
  only letters, numbers, and delimiters '_' or '.', as these are safe characters
  for column names in R. [This is a message, not a warning or an error]

estimating size factors

  Note: levels of factors in the design contain characters other than
  letters, numbers, '_' and '.'. It is recommended (but not required) to use
  only letters, numbers, and delimiters '_' or '.', as these are safe characters
  for column names in R. [This is a message, not a warning or an error]

estimating dispersions

gene-wise dispersion estimates

mean-dispersion relationship

  Note: levels of factors in the design contain characters other than

[1] "Activated memory B cell"


Joining with `by = join_by(pbmc_sample_id)`
converting counts to integer mode

Warning message in DESeqDataSet(se, design = design, ignoreRank):
“some variables in design formula are characters, converting to factors”
  Note: levels of factors in the design contain characters other than
  letters, numbers, '_' and '.'. It is recommended (but not required) to use
  only letters, numbers, and delimiters '_' or '.', as these are safe characters
  for column names in R. [This is a message, not a warning or an error]

estimating size factors

  Note: levels of factors in the design contain characters other than
  letters, numbers, '_' and '.'. It is recommended (but not required) to use
  only letters, numbers, and delimiters '_' or '.', as these are safe characters
  for column names in R. [This is a message, not a warning or an error]

estimating dispersions

gene-wise dispersion estimates

mean-dispersion relationship

  Note: levels of factors in the design contain characters other than

[1] "Adaptive NK cell"


...

Joining with `by = join_by(pbmc_sample_id)`
converting counts to integer mode

Warning message in DESeqDataSet(se, design = design, ignoreRank):
“some variables in design formula are characters, converting to factors”
  Note: levels of factors in the design contain characters other than
  letters, numbers, '_' and '.'. It is recommended (but not required) to use
  only letters, numbers, and delimiters '_' or '.', as these are safe characters
  for column names in R. [This is a message, not a warning or an error]

estimating size factors

  Note: levels of factors in the design contain characters other than
  letters, numbers, '_' and '.'. It is recommended (but not required) to use
  only letters, numbers, and delimiters '_' or '.', as these are safe characters
  for column names in R. [This is a message, not a warning or an error]

estimating dispersions

gene-wise dispersion estimates

mean-dispersion relationship

  Note: levels of factors in the design contain characters other than

[1] "cDC1"


Joining with `by = join_by(pbmc_sample_id)`
converting counts to integer mode

Warning message in DESeqDataSet(se, design = design, ignoreRank):
“some variables in design formula are characters, converting to factors”
  Note: levels of factors in the design contain characters other than
  letters, numbers, '_' and '.'. It is recommended (but not required) to use
  only letters, numbers, and delimiters '_' or '.', as these are safe characters
  for column names in R. [This is a message, not a warning or an error]

estimating size factors

  Note: levels of factors in the design contain characters other than
  letters, numbers, '_' and '.'. It is recommended (but not required) to use
  only letters, numbers, and delimiters '_' or '.', as these are safe characters
  for column names in R. [This is a message, not a warning or an error]

estimating dispersions

gene-wise dispersion estimates

mean-dispersion relationship

  Note: levels of factors in the design contain characters other than

[1] "pDC"


Joining with `by = join_by(pbmc_sample_id)`
converting counts to integer mode

Warning message in DESeqDataSet(se, design = design, ignoreRank):
“some variables in design formula are characters, converting to factors”
  Note: levels of factors in the design contain characters other than
  letters, numbers, '_' and '.'. It is recommended (but not required) to use
  only letters, numbers, and delimiters '_' or '.', as these are safe characters
  for column names in R. [This is a message, not a warning or an error]

estimating size factors

  Note: levels of factors in the design contain characters other than
  letters, numbers, '_' and '.'. It is recommended (but not required) to use
  only letters, numbers, and delimiters '_' or '.', as these are safe characters
  for column names in R. [This is a message, not a warning or an error]

estimating dispersions

gene-wise dispersion estimates

mean-dispersion relationship

  Note: levels of factors in the design contain characters other than

# Flu Y1 D0 VS D7---Flu Y2 D0 vs D7 - seperate BR1/BR2

# 

### Y1

In [ ]:
for (Cohort in c("BR1","BR2")){
meta_data_subset<-meta_data%>% filter(cohort.cohortGuid==Cohort,sample.visitName %in% c('Flu Year 1 Day 0','Flu Year 1 Day 7'))
bulk_list<-mclapply(meta_data_subset$pbmc_sample_id,function(x){

pbmc <-sceasy::convertFormat(paste0('/home/jupyter/BRI_Analysis/scRNA/h5_cleaned_by_sample/',x,'.h5ad'), from="anndata", to="seurat")
bulk <- Seurat:::PseudobulkExpression(object = pbmc, pb.method = 'aggregate',   group.by = "AIFI_L3",slot = 'counts')$RNA
colnames(bulk)<-paste0(x,":",colnames(bulk))
return(bulk)

},mc.cores=60)
    
for (celltype in str_extract(colnames(bulk_list[[1]]), "(?<=:).*")){
print(celltype)
matirx_list<-mclapply(bulk_list,function(bulk_matrix){

matching_columns <- grepl(celltype, colnames(bulk_matrix),, fixed = TRUE)
matching_columns<-colnames(bulk_matrix)[matching_columns]
bulk_matrix_single <- as.data.frame(bulk_matrix[, matching_columns, drop = FALSE])
return(bulk_matrix_single)
},mc.cores=60)

df_all<-do.call(cbind,matirx_list)
df_meta_data<-as.data.frame(colnames(df_all))
colnames(df_meta_data)<-'sample_name'
split_columns <- strsplit(df_meta_data$sample_name, ":")
df_meta_data$pbmc_sample_id <- sapply(split_columns, `[`, 1)
df_meta_data$AIFI_L3 <- sapply(split_columns, `[`, 2)
df_meta_data<-left_join(df_meta_data,meta_data_subset)
rownames(df_meta_data)<-df_meta_data$sample_name
selected_gene_list<-read.csv('filtered_gene_list.csv')
selGenes<-selected_gene_list %>% filter(AIFI_L3==celltype) %>% select(gene) %>% pull()  

dds <- DESeqDataSetFromMatrix(df_all[selGenes,], colData = df_meta_data, 
                              design = ~  sample.visitName+
                                          subject.biologicalSex+
                                          CMV.IgG.Serology.Result.Interpretation)
dds <- DESeq(dds,parallel=FALSE)

res <- data.frame(results(dds,contrast=c("sample.visitName", "Flu Year 1 Day 0", "Flu Year 1 Day 7")))
res$celltype<-celltype
res=res %>% arrange(padj) %>% mutate(Direction=case_when(log2FoldChange<0~'HigherInD7',log2FoldChange>0~'HigherInD0'))
res$Year<-"Y1"
res$cohort.cohortGuid<-Cohort  
write.csv(res,paste0("01C_DESeq2_D0_VS_D7_L3_by_Cohorts/",celltype,"_Y1_",Cohort,".csv"))

}
    

}

[1] "ASDC"


Joining with `by = join_by(pbmc_sample_id)`
converting counts to integer mode

Warning message in DESeqDataSet(se, design = design, ignoreRank):
“some variables in design formula are characters, converting to factors”
  Note: levels of factors in the design contain characters other than
  letters, numbers, '_' and '.'. It is recommended (but not required) to use
  only letters, numbers, and delimiters '_' or '.', as these are safe characters
  for column names in R. [This is a message, not a warning or an error]

estimating size factors

  Note: levels of factors in the design contain characters other than
  letters, numbers, '_' and '.'. It is recommended (but not required) to use
  only letters, numbers, and delimiters '_' or '.', as these are safe characters
  for column names in R. [This is a message, not a warning or an error]

estimating dispersions

gene-wise dispersion estimates

mean-dispersion relationship

  Note: levels of factors in the design contain characters other than

[1] "Activated memory B cell"


Joining with `by = join_by(pbmc_sample_id)`
converting counts to integer mode

Warning message in DESeqDataSet(se, design = design, ignoreRank):
“some variables in design formula are characters, converting to factors”
  Note: levels of factors in the design contain characters other than
  letters, numbers, '_' and '.'. It is recommended (but not required) to use
  only letters, numbers, and delimiters '_' or '.', as these are safe characters
  for column names in R. [This is a message, not a warning or an error]

estimating size factors

  Note: levels of factors in the design contain characters other than
  letters, numbers, '_' and '.'. It is recommended (but not required) to use
  only letters, numbers, and delimiters '_' or '.', as these are safe characters
  for column names in R. [This is a message, not a warning or an error]

estimating dispersions

gene-wise dispersion estimates

mean-dispersion relationship

  Note: levels of factors in the design contain characters other than

[1] "Adaptive NK cell"


...

Joining with `by = join_by(pbmc_sample_id)`
converting counts to integer mode

Warning message in DESeqDataSet(se, design = design, ignoreRank):
“some variables in design formula are characters, converting to factors”
  Note: levels of factors in the design contain characters other than
  letters, numbers, '_' and '.'. It is recommended (but not required) to use
  only letters, numbers, and delimiters '_' or '.', as these are safe characters
  for column names in R. [This is a message, not a warning or an error]

estimating size factors

  Note: levels of factors in the design contain characters other than
  letters, numbers, '_' and '.'. It is recommended (but not required) to use
  only letters, numbers, and delimiters '_' or '.', as these are safe characters
  for column names in R. [This is a message, not a warning or an error]

estimating dispersions

gene-wise dispersion estimates

mean-dispersion relationship

-- note: fitType='parametric', but the dispersion trend was not well 

[1] "CM CD8 T cell"


Joining with `by = join_by(pbmc_sample_id)`
converting counts to integer mode

Warning message in DESeqDataSet(se, design = design, ignoreRank):
“some variables in design formula are characters, converting to factors”
  Note: levels of factors in the design contain characters other than
  letters, numbers, '_' and '.'. It is recommended (but not required) to use
  only letters, numbers, and delimiters '_' or '.', as these are safe characters
  for column names in R. [This is a message, not a warning or an error]

estimating size factors

  Note: levels of factors in the design contain characters other than
  letters, numbers, '_' and '.'. It is recommended (but not required) to use
  only letters, numbers, and delimiters '_' or '.', as these are safe characters
  for column names in R. [This is a message, not a warning or an error]

estimating dispersions

gene-wise dispersion estimates

mean-dispersion relationship

  Note: levels of factors in the design contain characters other than

[1] "CMP cell"


Joining with `by = join_by(pbmc_sample_id)`


### Y2

In [ ]:
for (Cohort in c("BR1","BR2")){
meta_data_subset<-meta_data%>% filter(cohort.cohortGuid==Cohort,sample.visitName %in% c('Flu Year 2 Day 0','Flu Year 2 Day 7'))
bulk_list<-mclapply(meta_data_subset$pbmc_sample_id,function(x){

pbmc <-sceasy::convertFormat(paste0('/home/jupyter/BRI_Analysis/scRNA/h5_cleaned_by_sample/',x,'.h5ad'), from="anndata", to="seurat")
bulk <- Seurat:::PseudobulkExpression(object = pbmc, pb.method = 'aggregate',   group.by = "AIFI_L3",slot = 'counts')$RNA
colnames(bulk)<-paste0(x,":",colnames(bulk))
return(bulk)

},mc.cores=60)
    
for (celltype in str_extract(colnames(bulk_list[[1]]), "(?<=:).*")){
print(celltype)
matirx_list<-mclapply(bulk_list,function(bulk_matrix){

matching_columns <- grepl(celltype, colnames(bulk_matrix),, fixed = TRUE)
matching_columns<-colnames(bulk_matrix)[matching_columns]
bulk_matrix_single <- as.data.frame(bulk_matrix[, matching_columns, drop = FALSE])
return(bulk_matrix_single)
},mc.cores=60)

df_all<-do.call(cbind,matirx_list)
df_meta_data<-as.data.frame(colnames(df_all))
colnames(df_meta_data)<-'sample_name'
split_columns <- strsplit(df_meta_data$sample_name, ":")
df_meta_data$pbmc_sample_id <- sapply(split_columns, `[`, 1)
df_meta_data$AIFI_L3 <- sapply(split_columns, `[`, 2)
df_meta_data<-left_join(df_meta_data,meta_data_subset)
rownames(df_meta_data)<-df_meta_data$sample_name
selected_gene_list<-read.csv('filtered_gene_list.csv')
selGenes<-selected_gene_list %>% filter(AIFI_L3==celltype) %>% select(gene) %>% pull()  

dds <- DESeqDataSetFromMatrix(df_all[selGenes,], colData = df_meta_data, 
                              design = ~  sample.visitName+
                                          subject.biologicalSex+cohort.cohortGuid+
                                          CMV.IgG.Serology.Result.Interpretation)
dds <- DESeq(dds,parallel=FALSE)

res <- data.frame(results(dds,contrast=c("sample.visitName", "Flu Year 2 Day 0", "Flu Year 2 Day 7")))
res$celltype<-celltype
res=res %>% arrange(padj) %>% mutate(Direction=case_when(log2FoldChange<0~'HigherInD7',log2FoldChange>0~'HigherInD0'))
res$Year<-"Y2"
res$cohort.cohortGuid<-Cohort  
write.csv(res,paste0("01C_DESeq2_D0_VS_D7_L3_by_Cohorts/",celltype,"_Y2_",Cohort,".csv"))

}
    

}